In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import random
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
dout = open('review_data.pkl', 'rb')
data = pickle.load(dout)
dout.close()

In [28]:
trainDF = pd.DataFrame()
trainDF['text'] = data['reviewText']
trainDF['labels'] = data['overall']
data_sample = trainDF.sample(frac=0.06, replace=False)

In [126]:
def _vectorizers(which, df, what):
    
    svd = TruncatedSVD(n_components=1, n_iter=3, random_state=3)
    Cvec = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
    tfidf_vec = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
    
    if what == 'train':
        if which=='cv':
            
            cvec_train = Cvec.fit_transform(df['text'])
            svd_train = svd.fit_transform(cvec_train)
            penkiDF = pd.DataFrame(svd_train)
            penkiDF['Label'] = df['labels'].values
            return penkiDF

        if which=='tf':
            
            train_tf = tfidf_vec.fit_transform(df['text'])
            svd_train_tf = svd.fit_transform(train_tf)
            penkiDF_tf = pd.DataFrame(svd_train_tf)
            penkiDF_tf['Label'] = df['labels'].values
            return penkiDF_tf
        
    if what == 'test':
        if which=='cv':
            
            cvec_test = Cvec.fit_transform(df['test'])
            svd_test = svd.fit_transform(cvec_test)
            testDF = pd.DataFrame(svd_test)
            return testDF
        
        if which=='tf':
            
            test_tf = tfidf_vec.fit_transform(df['test'])
            svd_test_tf = svd.fit_transform(test_tf)
            test_tf = pd.DataFrame(svd_test_tf)
            return test_tf

In [66]:
which = 'cv' 
df = _vectorizers(which, data_sample)

In [61]:
def _labeling(df):
    
    lis = df['Label'].values
    final = np.zeros(len(lis))
    
    for i in range(len(lis)):
        if(lis[i] == 1 or lis[i] == 2):
            final[i] = 1
        if(lis[i] == 3):
            final[i] = 2
        if(lis[i] == 4 or lis[i] == 5):
            final[i] = 3
    
    df['Label'] = pd.DataFrame(final).values
    return df

In [67]:
# df1 = _labeling(df)

In [119]:
knn = KNeighborsClassifier(n_neighbors=5).fit(df1.iloc[:,0:30], df1['Label'])


In [73]:
# filename = 'knn.pkl'
# pickle.dump(knn, open(filename, 'wb'))

In [111]:
def predict_knn(model, test):
    return model.predict(test)


In [17]:
predict_knn(knn, X_test)

array([3, 3, 3, ..., 3, 3, 3])

In [127]:
test = ['This is an amazing product, i surely will recommend']
rev = pd.DataFrame(['This is an amazing product, i surely will recommend'], columns=['test'])
# rev['test'] = pd.DataFrame(test)
# _vectorizers('cv', rev1,'test')

In [128]:
rev
vecs = _vectorizers('cv', rev, 'test')

/Users/sudhigopal/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/truncated_svd.py:192: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


In [129]:
vecs


,0
0,3.0


In [130]:
predict_knn(knn, vecs)

ValueError: query data dimension must match training data dimension